In [11]:
## Import Libraries
import numpy as np
import pandas as pd
import copy
import glob

from tqdm.auto import trange

import torch as th
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch import Tensor
from torch.utils.data import DataLoader, SubsetRandomSampler, TensorDataset
import torch.optim as optim

from torchvision import datasets, transforms


import matplotlib.pyplot as plt

# from mpl_toolkits import mplot3d
from matplotlib.colors import ListedColormap

import seaborn as sns

from sklearn.decomposition import PCA, KernelPCA

from sklearn.metrics import adjusted_rand_score, accuracy_score, davies_bouldin_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV

from sklearn.cluster import KMeans, SpectralClustering
from sklearn.mixture import GaussianMixture
from sklearn.svm import SVC

# Used to save data into files
import pickle as pkl
import os

# Used to measure time
import time

In [12]:
custom_colors = [
    "#a6cee3",
    "#1f78b4",
    "#b2df8a",
    "#33a02c",
    "#fb9a99",
    "#e31a1c",
    "#fdbf6f",
    "#ff7f00",
    "#cab2d6",
    "#6a3d9a",
]
sns.set_palette(custom_colors)

ATOL = 0.05

In [13]:
folder_path = "./results/"  # Replace with the actual path to your folder
file_pattern = "alphas_df*.csv"

# Use glob to match files based on the pattern
data_files = glob.glob(f"{folder_path}/{file_pattern}")
print(data_files)

['./results/alphas_df_it_4.csv', './results/alphas_df_it_{iteration}.csv', './results/alphas_df_it_1.csv', './results/alphas_df_it_3.csv', './results/alphas_df_it_0.csv', './results/alphas_df_it_2.csv']


In [14]:
def plot_alphas(ax, batch_alpha_0, batch_alpha_1, batch_alpha_2):
    xrange = range(len(batch_alpha_0))
    ax.plot(xrange, batch_alpha_0, label="Alpha 0")
    ax.plot(xrange, batch_alpha_1, label="Alpha 1")
    ax.plot(xrange, batch_alpha_2, label="Alpha 2")
    # ax.set_xlabel("Number of seen batches")
    # ax.set_ylabel("Alpha", rotation=0, labelpad=20)
    ax.legend(loc="upper right")
    ax.set_ylim(0, 1)
    # ax.set_title(
    #     f"Alpha0: {round(batch_alpha_0[0].item(), 2)}, Alpha1: {round(batch_alpha_0[1].item(), 2)}, Alpha2: {round(batch_alpha_0[2].item(), 2)}",
    #     fontweight="bold",
    # )
    return ax

In [15]:
df = pd.read_csv(data_files[0])

condition = (df["lambda"] == 0.015) & (df["iteration"] == 3)
print(len(df.loc[condition, "alpha0"]))

46900


In [16]:
data_files = ["./results/alphas_df_0.csv"]

lambda_range = np.arange(start=0, stop=0.051, step=0.005)

for f_counter, file in enumerate(data_files):
    for counter, lam in enumerate(lambda_range):
        df = pd.read_csv(file)

        fig, ax = plt.subplots(2, 3, figsize=(15, 10), dpi=200)
        ax = ax.flatten()
        for i in range(6):
            condition = (df["lambda"] == lam) & (df["iteration"] == i)
            ax[i] = plot_alphas(
                ax[i],
                df.loc[condition, "alpha0"],
                df.loc[condition, "alpha1"],
                df.loc[condition, "alpha2"],
            )
            ax[i].legend(loc="upper center")
        ax[0].set_xlabel("Number of seen batches", fontweight="bold")
        ax[0].set_ylabel("Alpha", rotation=0, labelpad=20, fontweight="bold")
        fig.suptitle(
            f"Alpha values for lambda = {round(lam, 3)}", size=20, fontweight="bold"
        )
        fig.subplots_adjust(top=0.90)
        # plt.legend(loc='upper center')
        plt.savefig(f"./alphas_plots/alphas_{f_counter}_{counter}.png")

/tmp/ipykernel_41351/196667087.py:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(2, 3, figsize=(15, 10), dpi=200)


In [17]:
# Showing the available data files with name alphas_df*.csv
print(f"Number of data files: {len(data_files)}")

Number of data files: 6


In [18]:
for i, file in enumerate(data_files):
    print(i, file)

0 ./results/alphas_df_it_4.csv
1 ./results/alphas_df_it_{iteration}.csv
2 ./results/alphas_df_it_1.csv
3 ./results/alphas_df_it_3.csv
4 ./results/alphas_df_it_0.csv
5 ./results/alphas_df_it_2.csv


In [19]:
# Initialize an empty list to store DataFrames
dfs_to_concat = []

for i, file in enumerate(data_files):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file)

    # Extract unique lambda values
    lambda_range = df["lambda"].unique()

    # Initialize an empty DataFrame for this file
    df_for_file = pd.DataFrame(
        columns=["df", "lambda", "alpha0", "alpha1", "alpha2", "total"]
    )

    # Iterate over unique lambda values
    for Lambda in lambda_range:
        df_lambda = df[df["lambda"] == Lambda].groupby("iteration").tail(1)

        # Calculate percentage of alphas that go to zero
        percentages_lambda = {}
        for col in ["alpha0", "alpha1", "alpha2"]:
            percentage_zero = np.isclose(df_lambda[col], 0, atol=ATOL).mean() * 100
            percentages_lambda[col] = percentage_zero

        # Add a row for this lambda to the DataFrame
        df_for_lambda = pd.DataFrame(
            {
                "df": [i],
                "lambda": [Lambda],
                "alpha0": [percentages_lambda.get("alpha0", np.nan)],
                "alpha1": [percentages_lambda.get("alpha1", np.nan)],
                "alpha2": [percentages_lambda.get("alpha2", np.nan)],
            }
        )
        df_for_file = pd.concat([df_for_file, df_for_lambda], ignore_index=True)

    # Calculate the total percentage for this file
    alphacols = ["alpha0", "alpha1", "alpha2"]
    df_for_file["total"] = df_for_file[alphacols].mean(axis=1)

    # Add the DataFrame for this file to the list
    dfs_to_concat.append(df_for_file)

# Concatenate all DataFrames into the final result
percentages_df = pd.concat(dfs_to_concat, ignore_index=True)

/tmp/ipykernel_41351/725415162.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_for_file = pd.concat([df_for_file, df_for_lambda], ignore_index=True)
/tmp/ipykernel_41351/725415162.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_for_file = pd.concat([df_for_file, df_for_lambda], ignore_index=True)
/tmp/ipykernel_41351/725415162.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude 

In [20]:
print(percentages_df.loc[percentages_df["df"] == 1])

   df  lambda      alpha0      alpha1      alpha2       total
11  1   0.000   33.333333   33.333333   16.666667   27.777778
12  1   0.005   16.666667   50.000000   33.333333   33.333333
13  1   0.010   50.000000   33.333333   33.333333   38.888889
14  1   0.015   50.000000   66.666667   83.333333   66.666667
15  1   0.020  100.000000   66.666667   66.666667   77.777778
16  1   0.025   33.333333   50.000000   50.000000   44.444444
17  1   0.030  100.000000  100.000000  100.000000  100.000000
18  1   0.035  100.000000  100.000000   50.000000   83.333333
19  1   0.040  100.000000  100.000000  100.000000  100.000000
20  1   0.045  100.000000  100.000000  100.000000  100.000000
21  1   0.050  100.000000  100.000000  100.000000  100.000000


In [21]:
percentages_df["df"].unique()

array([0, 1, 2, 3, 4, 5], dtype=object)

In [22]:
fig, ax = plt.subplots(figsize=(10, 5), dpi=200)
for df in percentages_df["df"].unique():
    ax.plot(
        percentages_df[percentages_df["df"] == df]["lambda"],
        percentages_df[percentages_df["df"] == df]["total"],
        marker="o",
        label=df,
    )
ax.set_xlabel(r"$\lambda$", fontweight="bold", fontsize=14)
ax.set_ylabel(
    r'% of $\alpha$' + '\nthat go to zero',
    rotation=0,
    labelpad=70,
    fontweight="bold",
    fontsize=14,
)
# ax.set_title(
#     "Percentage of alphas that go to zero for different lambda values",
#     fontweight="bold",
#     size=20,
# )
ax.set_yticks(np.arange(0, 101, 20))
ax.legend(title="Data file")
ax.grid(True, axis="y")
plt.savefig("./plots/alphas_total.png")

Dati molto sospetti zio pera... raccogline altri e vedi che succede. Se il pattern persiste cercare di capirne la causa

idea: non c'era lo shuffling del dataset

## Distribution of converged values

In [23]:
# Getting all the converged values
for file in data_files:
    final_values = pd.DataFrame(columns=["lambda", "alpha0", "alpha1", "alpha2"])
    df = pd.read_csv(file)
    lambda_range = df["lambda"].unique()

    for Lambda in lambda_range:
        df_lambda = df[df["lambda"] == Lambda].groupby("iteration").tail(1)
        final_values = pd.concat(
            [final_values, df_lambda[["lambda", "alpha0", "alpha1", "alpha2"]]],
            ignore_index=True,
        )

print(final_values)

/tmp/ipykernel_41351/1649795383.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_values = pd.concat(
/tmp/ipykernel_41351/1649795383.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_values = pd.concat(
/tmp/ipykernel_41351/1649795383.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclud

    lambda  alpha0  alpha1  alpha2
0     0.00  0.9977  0.0106  0.0233
1     0.00  0.9929  0.0000  0.0006
2     0.00  0.4587  0.0014  0.0005
3     0.00  0.9919  0.0439  0.0030
4     0.00  0.0000  0.9921  0.0002
..     ...     ...     ...     ...
61    0.05  0.0000  0.0000  0.0000
62    0.05  0.0012  0.0003  0.0014
63    0.05  0.0016  0.0000  0.0000
64    0.05  0.0004  0.0000  0.0005
65    0.05  0.0003  0.0000  0.0005

[66 rows x 4 columns]


/tmp/ipykernel_41351/1649795383.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_values = pd.concat(


In [24]:
# Melt the DataFrame to have a single column for values and another for variable names
melted_values = final_values.melt(var_name="variable", value_name="value", id_vars="lambda")
print(melted_values)

     lambda variable   value
0      0.00   alpha0  0.9977
1      0.00   alpha0  0.9929
2      0.00   alpha0  0.4587
3      0.00   alpha0  0.9919
4      0.00   alpha0  0.0000
..      ...      ...     ...
193    0.05   alpha2  0.0000
194    0.05   alpha2  0.0014
195    0.05   alpha2  0.0000
196    0.05   alpha2  0.0005
197    0.05   alpha2  0.0005

[198 rows x 3 columns]


In [25]:

g = sns.FacetGrid(
    melted_values,
    row="variable",
    hue="variable",
    palette=custom_colors,
    margin_titles=True
)
g.map(sns.stripplot, "value")
g.set(xlim=(0, 1))
g.set_titles("{value}", fontweight="bold")

g.set_ylabels("Count", rotation=0, labelpad=20, fontweight="bold")
g.set_xlabels(r"$\alpha$", fontweight="bold", fontsize=14)
g.fig.set_size_inches(10, 5.5)
g.fig.set_dpi(200)
g.fig.tight_layout()
plt.show()  


# fig, ax = plt.subplots(figsize=(10, 5), dpi=100)
# sns.stripplot(data=final_values, x="alpha0", ax=ax, label="alpha0")
# sns.stripplot(data=final_values, x="alpha1", ax=ax, label="alpha1")
# sns.stripplot(data=final_values, x="alpha2", ax=ax, label="alpha2")
# ax.legend()

/home/pettepiero/local/envs/atml/lib/python3.11/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/pettepiero/local/envs/atml/lib/python3.11/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/pettepiero/local/envs/atml/lib/python3.11/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/pettepiero/local/envs/atml/lib/python3.11/site-packages/seaborn/axisgrid.py:712: UserWarning: Using the stripplot function without specifying `order` is likely to produc

In [26]:
bin_edges = [i / 10 for i in range(11)]

g = sns.FacetGrid(
    melted_values,
    row="variable",
    col="lambda",
    hue="variable",
    palette=custom_colors,
    margin_titles=True
)
g.map(sns.histplot, "value", bins=bin_edges)
g.set(xlim=(0, 1))
g.set_titles("{lambda}", fontweight="bold")

g.set_ylabels("Count", rotation=0, labelpad=20, fontweight="bold")
g.set_xlabels("Alpha", fontweight="bold")
g.fig.set_size_inches(15, 5.5)
g.fig.set_dpi(200)
g.fig.tight_layout()
plt.show()

/home/pettepiero/local/envs/atml/lib/python3.11/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/pettepiero/local/envs/atml/lib/python3.11/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/pettepiero/local/envs/atml/lib/python3.11/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/pettepiero/local/envs/atml/lib/python3.11/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future versi

: 